# Xarray

This document describes the basics of the xarray library, which provides the datatypes most commonly used when working with the Open Data Cube.

## Background
`Xarray` is a python library which simplifies working with labelled multi-dimension arrays. `Xarray` introduces labels in the forms of dimensions, coordinates and attributes on top of raw `numpy` arrays, allowing for more intitutive and concise development.

First we will load data from the Data Cube to have some example data to work with.

In [1]:
import datacube

# Allow importing of our utilities.
import sys
sys.path.append('../../Scripts')
from deafrica_datahandling import load_ard

In [2]:
dc = datacube.Datacube(app="intro_to_xarray")

In [3]:
# Dar es Salaam, Tanzania - 2018
query = dict(dc=dc,
             min_gooddata=0.7,
             x=(39.20, 39.37),
             y=(-6.90, -6.70),
             time=("2018-01-01", "2018-12-31"),
             output_crs="EPSG:4326",
             resolution=(-0.00027, 0.00027),
             group_by='solar_day')

In [4]:
landsat_ds = load_ard(products=["ls7_usgs_sr_scene", "ls8_usgs_sr_scene"],
                      **query)

Using pixel quality parameters for USGS Collection 1
Finding datasets
    ls7_usgs_sr_scene
    ls8_usgs_sr_scene
Counting good quality pixels for each time step
Filtering to 4 out of 41 time steps with at least 70.0% good quality pixels
Applying pixel quality/cloud mask
Loading 4 time steps


As we saw in the document regarding loading data, the output of `load_ard()` - `landsat_ds` - is an `xarray.Dataset` object, as can be seen below.

In [5]:
landsat_ds

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630, time: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-31T07:31:54.636976 ... 2018-12-01T07:31:56.186494
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (time, latitude, longitude) float32 nan nan nan ... 443.0 704.0
    green        (time, latitude, longitude) float32 nan nan nan ... 534.0 857.0
    red          (time, latitude, longitude) float32 nan nan nan ... 594.0 969.0
    nir          (time, latitude, longitude) float32 nan nan ... 1245.0 1967.0
    swir1        (time, latitude, longitude) float32 nan nan ... 1355.0 2326.0
    swir2        (time, latitude, longitude) float32 nan nan ... 991.0 1689.0
    radsat_qa    (time, latitude, longitude) float32 nan nan nan ... 0.0 0.0 0.0
    pixel_qa     (time, latitude, longitude) uint16 328 328 328 ... 322 322 322
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

### Interpreting the resulting `xarray.Dataset`
The variable `landsat_ds` contains all data that matched the query parameters (spatial, temporal, product, etc.) inputted into `load_ard()`.

All `xarray.Dataset` objects have these properties:

* Dimensions: These note the sizes of the data along each dimension (the shape).
* Coordinates: These allow indexing of the data by coordinate values.
* Data variables: These are what contain the data. For data loaded from the Data Cube, each `measurement` specified in the query will be a data variable in the returned `xarray.Dataset`.
* Attributes: These contain miscellaneous information about the data.

Here is a description of the significance of these properties for this data:

*Dimensions*

* Identifies the number of timesteps returned in the search (`time`) as well as the number of pixels in the x and y dimensions of the data query. In this case, using a CRS of EPSG:4326, the x and y dimensions are called `longitude` and `latitude`, respectively. However, for many coordinate systems, they will be called `x` and `y`.

*Coordinates* 

* `time` identifies the date attributed to each returned timestep.
* `longitude` and `latitude` are the coordinates for each pixel within the spatial bounds of your query.

*Data variables*

* These are the measurements available for the nominated product. 
For every date (`time`) returned by the query, the measured value at each pixel (`latitude`, `longitude`) is returned as an array for each measurement.
Each data variable is itself an `xarray.DataArray` object (see below). So this data really has 4 dimensions - 2 for space (`latitude`, `longitude`), 1 for time (`time`), and 1 for the data variables (`red`, `green`, `blue`, ...).

*Attributes*

* `crs` identifies the coordinate reference system (CRS) of the loaded data. 



### Inspecting an individual `xarray.DataArray`
The `xarray.Dataset` object we loaded above is a collection of individual `xarray.DataArray` objects that hold the actual data for each data variable/measurement. 
For example, all measurements listed under _Data variables_ above (e.g. `blue`, `green`, `red`, `nir`, `swir1`, `swir2`) are `xarray.DataArray` objects.

We can inspect the data in these `xarray.DataArray` objects for an `xarray.Dataset` named `ds` using either of the following syntaxes:
```
ds["measurement_name"]
```
or:
```
ds.measurement_name
```

Being able to access data from individual data variables/measurements allows us to manipulate and analyse data from individual satellite bands or specific layers in a dataset. 
For example, we can access data from the near infra-red satellite band (i.e. `nir`):

In [6]:
landsat_ds.nir

<xarray.DataArray 'nir' (time: 4, latitude: 742, longitude: 630)>
array([[[  nan,   nan,   nan, ...,  148.,  170.,  219.],
        [  nan,   nan,   nan, ...,  141.,  167.,  224.],
        [  nan,   nan,   nan, ...,  145.,  174.,  232.],
        ...,
        [  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [  nan,   nan,   nan, ...,   nan,   nan,   nan]],

       [[3376., 2545., 2431., ...,  223.,  207.,  219.],
        [2677., 2432., 2337., ...,  205.,  206.,  208.],
        [2525., 2405., 2720., ...,  201.,  205.,  210.],
        ...,
        [3119., 2948., 2614., ..., 3014., 3091., 3032.],
        [3436., 3416., 2806., ..., 2905., 2676., 3132.],
        [3022., 3487., 3183., ..., 2851., 2529., 3146.]],

       [[2656., 2235., 2235., ...,  238.,  238.,  238.],
        [2445., 2235., 2319., ...,  238.,  280.,  238.],
        [  nan,   nan,   nan, ...,  238.,  238.,  238.],
        ...,
        [2492., 2281., 2197., ..., 2446., 2488., 2320.],
        [2661., 2451., 2027., ..., 2403., 2446., 2362.],
        [2661., 2704., 2366., ..., 2403., 2362., 2320.]],

       [[  nan,   nan,   nan, ...,  153.,  153.,  150.],
        [  nan,   nan,   nan, ...,  124.,  140.,  143.],
        [  nan,   nan,   nan, ...,  143.,  148.,  147.],
        ...,
        [3699., 3109., 2804., ..., 2648., 2992., 3104.],
        [3294., 3304., 3058., ..., 1725., 2066., 2820.],
        [2829., 3267., 3399., ..., 1592., 1245., 1967.]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-31T07:31:54.636976 ... 2018-12-01T07:31:56.186494
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Attributes:
    units:         reflectance
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

Note that the object header informs us that it is an `xarray.DataArray` containing data for the `nir` satellite band. 

Like an `xarray.Dataset`, the array also includes information about the data's dimensions, coordinates and attributes**.

> **Note**: For a more in-depth introduction to `xarray` data structures, refer to the [official xarray documentation](http://xarray.pydata.org/en/stable/data-structures.html)

## Indexing

Indexing data in an `xarray.Dataset` or `xarray.DataArray` can be done in 2 ways: integer indexing and label indexing.

### Integer indexing
Integer indexing is the selection of data by coordinate **indexes**. The first index in a coordinate array (e.g. `time`) is 0. The next index is 1, and so on. In xarray, this is achieved with the [`isel()`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.isel.html) method.

Here is an example of integer indexing with the `time` dimension. The following code selects the first time slice. Notice that there is no `time` dimension in the output. There is still a `time` coordinate, but it is a single value.

In [7]:
landsat_ds.isel(time=0)

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630)
Coordinates:
    time         datetime64[ns] 2018-01-31T07:31:54.636976
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    green        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    red          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    nir          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    swir1        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    swir2        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    radsat_qa    (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    pixel_qa     (latitude, longitude) uint16 328 328 328 328 ... 480 480 480
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

Multiple coordinate values of a dimension can be selected with `slice`. The lower value is inclusive and the upper value is exclusive, so `slice(0,3)` selects the first 3 elements.

In [8]:
landsat_ds.isel(time=slice(0,3))

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630, time: 3)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-31T07:31:54.636976 ... 2018-07-18T07:31:05.637464
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (time, latitude, longitude) float32 nan nan nan ... 718.0 690.0
    green        (time, latitude, longitude) float32 nan nan nan ... 960.0 895.0
    red          (time, latitude, longitude) float32 nan nan ... 1163.0 1076.0
    nir          (time, latitude, longitude) float32 nan nan ... 2362.0 2320.0
    swir1        (time, latitude, longitude) float32 nan nan ... 3474.0 3279.0
    swir2        (time, latitude, longitude) float32 nan nan ... 2769.0 2523.0
    radsat_qa    (time, latitude, longitude) float32 nan nan nan ... 0.0 0.0 0.0
    pixel_qa     (time, latitude, longitude) uint16 328 328 328 328 ... 66 66 66
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

### Label indexing
Label indexing is the selection of data by coordinate **values** (or "labels"). Coordinate values for x and y dimensions are numbers. Coordiante values for the `time` dimension have a certain time datatype, though we can use date strings of the format `YYYY-MM-DD` to index by time. In xarray, this is achieved with the [`sel()`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.sel.html) method.

Here is an example of label indexing with the `time` dimension. The following code selects the first time slice.

In [9]:
landsat_ds.sel(time="2018-01-31")

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-31T07:31:54.636976
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    green        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    red          (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    nir          (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    swir1        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    swir2        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    radsat_qa    (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    pixel_qa     (time, latitude, longitude) uint16 328 328 328 ... 480 480 480
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

Notice that, in this case, there is a `time` dimension in the output. Additionally, the `time` coordinate is a single value, but that value is within an array (indicated by its **bold** font).

In other words, the data is still 3D. To achieve the same result as we did when using `isel()`, call [`squeeze()`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.squeeze.html), which removes dimensions of length 1.

In [10]:
landsat_ds.sel(time="2018-01-31").squeeze()

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630)
Coordinates:
    time         datetime64[ns] 2018-01-31T07:31:54.636976
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    green        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    red          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    nir          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    swir1        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    swir2        (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    radsat_qa    (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    pixel_qa     (latitude, longitude) uint16 328 328 328 328 ... 480 480 480
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

Label indexing also supports `slice`.

In [11]:
landsat_ds.sel(time=slice("2018-01-31", "2018-07-18"))

<xarray.Dataset>
Dimensions:      (latitude: 742, longitude: 630, time: 3)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-31T07:31:54.636976 ... 2018-07-18T07:31:05.637464
  * latitude     (latitude) float64 -6.7 -6.7 -6.7 -6.701 ... -6.899 -6.9 -6.9
    spatial_ref  int32 4326
  * longitude    (longitude) float64 39.2 39.2 39.2 39.2 ... 39.37 39.37 39.37
Data variables:
    blue         (time, latitude, longitude) float32 nan nan nan ... 718.0 690.0
    green        (time, latitude, longitude) float32 nan nan nan ... 960.0 895.0
    red          (time, latitude, longitude) float32 nan nan ... 1163.0 1076.0
    nir          (time, latitude, longitude) float32 nan nan ... 2362.0 2320.0
    swir1        (time, latitude, longitude) float32 nan nan ... 3474.0 3279.0
    swir2        (time, latitude, longitude) float32 nan nan ... 2769.0 2523.0
    radsat_qa    (time, latitude, longitude) float32 nan nan nan ... 0.0 0.0 0.0
    pixel_qa     (time, latitude, longitude) uint16 328 328 328 328 ... 66 66 66
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

## Learn more

More information about `xarray` data structures, functions, and more can be found [here](http://xarray.pydata.org/en/stable/).

See the ["How do I..."](http://xarray.pydata.org/en/stable/howdoi.html) xarray webpage for guidance about what xarray methods to use in different scenarios.

This [external notebook](https://rabernat.github.io/research_computing/xarray.html) introduces more uses of xarray and may help you advance your skills further.